# EN1 Fall 2019: Project 5

Here are some experimental codes you can use for **Project 5: Color and Distance Sensors**.

SPIKE Prime setup for these examples are: motor plugged into Port A, Distance Sensor plugged into Port C, and Color Sensor plugged into Port E.

## Code 1: Setting Up SPIKE Prime

(This is the same as Project 4.  Just to make sure things are working!)

Use this to check that SPIKE Prime is connecting properly.  This code:
- opens up a serial connection
- checks battery
- beeps
- checks a Force Sensor plugged into Port D
- then closes the connection

If it opens serial, beeps, and closes but ***DOES NOT*** get good readings of the battery or Force Sensor, then there is a communication error between the SPIKE Prime and Jupyter Notebooks (reading values BACK).  Please try again!

In [ ]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE

mySPIKE = SPIKE()
print(mySPIKE.ListDevices())
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

val = mySPIKE.GetValue('hub.battery.voltage()')
print('SPIKE Prime Voltage:',val,'millivolts')

print('Beeeeeep!')
mySPIKE.SendCommand('hub.sound.beep()')

val = mySPIKE.GetValue('hub.port.C.device.get()')
print('Distance:',val,'cm')
val = mySPIKE.GetValue('hub.port.E.device.get()')
print('Color:',val)

mySPIKE.CloseSerial()
print('Closed Connection')

## Code 2: Read Color Value

This code is in three parts:
1. Setting up and opening serial connection
2. Reading the Color Sensor
3. Closing the serial connection (always good programming)

### Reading the Color Sensor

The color sensor has three modes:
- Mode 0: Color Mode (returns an integer representing the color, or `None` if no color detected)
- Mode 1: Reflected Light (returns percent reflected light off a surface, 0% to 100%)
- Mode 2: Ambient Light (returns percent ambient light entering sensor, 0% to 100%)

Note: when you "switch modes" it takes a second for the sensor to adjust before you can read it.  So in the code below you'll see a slight delay added after switching modes and reading value in order to give sensor time to adjust.

If the sensor doesn't read anything, it returns the Python `None` type.  If it does return a value, it is a `list` of one value: the integer (`int`) value of the reading. It also might return something else (e.g. Error). So, need to check the type of value before processing.

Tons of info on `types`: https://docs.python.org/3.7/library/stdtypes.html

### Color Mode Info

Here are a few of the color numbers:
- `4`: Light Blue
- `5`: Green
- `7`: Yellow
- `10`: White

You can experiment to determine other colors it detects and their associated values.

In [6]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE
mySPIKE = SPIKE()
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

Connected to: /dev/ttyACM0


In [8]:
# here is the code to read the Color Sensor
time_delay = 0.08

# test Color Sensor (Port E) mode 0 (color mode)
mySPIKE.SendCommand('hub.port.E.device.mode(0)')
time.sleep(time_delay)
val = mySPIKE.GetValue('hub.port.E.device.get()') # get value of Color Sensor
time.sleep(time_delay)
if type(val) == None:
    color = None # should already be this
elif type(val) == list:
    color = val[0] # convert value to integer
else:
    color = 'Error (' + str(val) + ')' # set to string
print('Color in Mode 0 (color):', color)

# test Color Sensor (Port E) mode 1 (reflective mode)
mySPIKE.SendCommand('hub.port.E.device.mode(1)')
time.sleep(time_delay)
val = mySPIKE.GetValue('hub.port.E.device.get()') # get value of Color Sensor
time.sleep(time_delay)
if type(val) == None:
    color = None # should already be this
elif type(val) == list:
    color = val[0] # convert value to integer
else:
    color = 'Error (' + str(val) + ')' # set to string
print('Color in Mode 1 (reflected):', color)

# test Color Sensor (Port E) mode 2 (ambient mode)
mySPIKE.SendCommand('hub.port.E.device.mode(2)')
time.sleep(time_delay)
val = mySPIKE.GetValue('hub.port.E.device.get()') # get value of Color Sensor
time.sleep(time_delay)
if type(val) == None:
    color = None # should already be this
elif type(val) == list:
    color = val[0] # convert value to integer
else:
    color = 'Error (' + str(val) + ')' # set to string
print('Color in Mode 2 (abient):', color)


Color in Mode 0 (color): 4
Color in Mode 1 (reflected): 66
Color in Mode 2 (abient): 0


In [5]:
mySPIKE.CloseSerial()
print('Closed Connection')

Closed Connection


## Code 3: Color Sensor Lookup

Here is a helper function for looking up the color and returning a string.

If the color isn't detected (python `None`) it returns the string `'None'` instead.

In [ ]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE
mySPIKE = SPIKE()
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

In [ ]:
# color helper function; import once
def lookup_color(mySPIKE, port):
    time_delay = 0.08 # how long to wait
    # color is mode(0) for Color Sensor
    mySPIKE.SendCommand('hub.port.' + str(port) + '.device.mode(0)')
    time.sleep(time_delay) # wait after switching mode
    val = mySPIKE.GetValue('hub.port.' + str(port) + '.device.get()') # get value of Color Sensor
    if type(val) == None:
        color = 'None' # note returning a STRING version of 'None'
    elif type(val) == list:
        color = val[0] # convert value to integer
        # now convert that integer into a string representing the color
        if color == 4:
            color = 'Light Blue'
        elif color == 5:
            color = 'Green'
        elif color == 7:
            color = 'Yellow'
        elif color == 10:
            color = 'White'
        else:
            # default case
            color = 'Unknown Color'
        pass
    else:
        # error, still return 'None'
        color = 'None' # set to string
    return color

In [ ]:
# read color in a loop
# - read 10 times, pausing 2-seconds each time
time_delay = 2
print('Reading color 10 times...')
for i in range(10):
    color = lookup_color(mySPIKE, 'E')
    print('Color is:', color)
    # do something based on color
    if color == 'Green':
        print(' - GREEN MEANS GOOD TO GO!')
    time.sleep(time_delay) # delay before next color read

In [ ]:
mySPIKE.CloseSerial()
print('Closed Connection')

## Code 4: Reading the Distance Sensor

This code is in three parts:
1. Setting up and opening serial connection
2. Reading the Distance Sensor
3. Closing the serial connection (always good programming)

Note: the Distance Sensor has different modes:
- Mode 0: good for (???)
- Mode 1: good for (???)
- Mode 2: good for (???)

Note: when you "switch modes" it takes a second for the sensor to adjust before you can read it.  So in the code below you'll see a slight delay added after switching modes and reading value in order to give sensor time to adjust.

If the sensor doesn't read anything, it returns the Python `None` type.  If it does return a value, it is a `list` of one value: the integer (`int`) value of the reading. It also might return something else (e.g. Error). So, need to check the type of value before processing.

Tons of info on `types`: https://docs.python.org/3.7/library/stdtypes.html

In [ ]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE
mySPIKE = SPIKE()
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

In [ ]:
# here is the code to read the Distance Sensor
time_delay = 0.08

# test Distance Sensor (Port C) mode 0
mySPIKE.SendCommand('hub.port.C.device.mode(0)')
time.sleep(time_delay)
val = mySPIKE.GetValue('hub.port.C.device.get()') # get value of Distance Sensor
time.sleep(time_delay)
if type(val) == None:
    dist = None # should already be this
elif type(val) == list:
    dist = val[0] # convert value to integer
else:
    dist = 'Error (' + str(val) + ')' # set to string
print('Distance in Mode 0:', dist, 'cm')

# test Distance Sensor (Port C) mode 1
mySPIKE.SendCommand('hub.port.C.device.mode(1)')
time.sleep(time_delay)
val = mySPIKE.GetValue('hub.port.C.device.get()') # get value of Distance Sensor
time.sleep(time_delay)
if val == None:
    dist = None
elif type(val) == list:
    dist = val[0] # convert value to integer
else:
    dist = 'Error (' + str(val) + ')' # set to string
print('Distance in Mode 1:', dist, 'cm')

# test Distance Sensor (Port C) mode 2
mySPIKE.SendCommand('hub.port.C.device.mode(2)')
time.sleep(time_delay)
val = mySPIKE.GetValue('hub.port.C.device.get()') # get value of Distance Sensor
time.sleep(time_delay)
if val == None:
    dist = None
elif type(val) == list:
    dist = val[0] # convert value to integer
else:
    dist = 'Error (' + str(val) + ')' # set to string
print('Distance in Mode 2:', dist, 'cm')

In [ ]:
mySPIKE.CloseSerial()
print('Closed Connection')

## Code 5: Average Distance (across modes)

Now we've created a helper function (`avg_sensor()`) that does the reading/processing of the Distance Sensor reading for us.  It checks each mode, stores the values, and averages them together.  If none of them return value data, it returns `int(-1)` so we can be sure to *always* be getting a value (and if `>= 0`, we know a valid value!).

Then we read the sensor in a loop so we can test it out.

In [ ]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE
mySPIKE = SPIKE()
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

In [ ]:
# this is the function that does the averaging
def avg_sensor(mySPIKE, port, modes):
    time_delay = 0.08 # how long to wait
    average = 0
    num_data_points = 0
    for mode in modes:
        # code that sends the new mode value to the right port
        mySPIKE.SendCommand('hub.port.' + str(port) + '.device.mode(' + str(mode) + ')')
        time.sleep(time_delay)
        val = mySPIKE.GetValue('hub.port.' + str(port) + '.device.get()')
        if type(val) == list:
            sensor_data = val[0]
            if type(sensor_data) == int:
                average = average + sensor_data
                num_data_points = num_data_points + 1
        time.sleep(time_delay)
    if num_data_points > 0:
        return average/num_data_points
    else:
        return -1 # didn't get any valid data, return -1

In [ ]:
# take 10 readings, every 1/2 second: so five seconds of data
for i in range(10):
    current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
    print('Distance Sensor:', current_val, 'cm')
    time.sleep(0.5)

In [ ]:
mySPIKE.CloseSerial()
print('Closed Connection')

# MicroPython Code

If you are running MicroPython code directly on the SPIKE Prime (not in Jupyter Notebook), then here are the above examples in MicroPython.

## Code 6: Read Color Sensor (MicroPython Code)

NOTE: must run on SPIKE Prime (Not in Jupyter Notebook)

This code reads Mode 0 (Color Mode) and displays color to the Hub's LED screen

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
# This code reads the Color Sensor (on Port E) and displays value to console
import time
import hub
hub.sound.beep() # make sure working
# set mode
hub.port.E.device.mode(0)
hub.display.show('Color:', fade=4) # color mode
time.sleep(3)
val = hub.port.E.device.get()
print(val) # print to console (debugging)
if type(val) == None:
    color = None # should already be this
elif type(val) == list:
    color = val[0] # convert value to integer
else:
    color = 'Error (' + str(val) + ')' # set to string
hub.display.show(str(color), fade=4) # show color

## Code 7: Lookup Color Name (MicroPython Code)

NOTE: must run on SPIKE Prime (Not in Jupyter Notebook)

Defines a function to help with color detection (converts number to name).

Then calls that in a loop displaying detected color to the Hub's LED screen.

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
# this just defines a function; doesn't actually "do" anything
# only need to input/run this once, then the next cell of code will call it
def lookup_color(port):
    time_delay = 0.08 # how long to wait
    # color is mode(0) for Color Sensor
    eval('hub.port.' + str(port) + '.device.mode(0)')
    time.sleep(time_delay) # wait after switching mode
    val = eval('hub.port.' + str(port) + '.device.get()') # get value of Color Sensor
    if type(val) == None:
        color = 'None' # note returning a STRING version of 'None'
    elif type(val) == list:
        color = val[0] # convert value to integer
        # now convert that integer into a string representing the color
        if color == 4:
            color = 'Light Blue'
        elif color == 5:
            color = 'Green'
        elif color == 7:
            color = 'Yellow'
        elif color == 10:
            color = 'White'
        else:
            # default case
            color = 'Unknown Color'
        pass
    else:
        # error, still return 'None'
        color = 'None' # set to string
    return color

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
# This calls the lookup_color() function 10 times (once every 3 seconds)
# and displays the name of the color detected each time
import time
import hub
time_delay = 3
for i in range(10):
    hub.display.show(lookup_color('E'), fade=4)
    time.sleep(time_delay)

## Code 8: Read Distance Sensor (MicroPython Code)

NOTE: must run on SPIKE Prime (Not in Jupyter Notebook)

Reads a Distance Sensor plugged into Port C.  (Reads three times: for modes 0, 1, and 2.)

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
# In a loop, this makes a decision based on the Force Sensor value
# - run motor if force is strong
import time
import hub
time_delay = 0.08
modes = [0, 1, 2]
for mode in modes:
    hub.port.C.device.mode(mode) # switch mode
    time.sleep(time_delay)
    val = hub.port.C.device.get() # read value
    print('Mode',mode,', Val',val) # print to console

## Code 9: Read Average Distance (MicroPython Code)

NOTE: must run on SPIKE Prime (Not in Jupyter Notebook)

Two Code Cells:
- first is a helper function that reads the distance multiple times (based on list of modes passed in)
- the second calls this function 10 times (once every 1/2 second)

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
def avg_sensor(port, modes):
    time_delay = 0.08 # how long to wait
    average = 0
    num_data_points = 0
    for mode in modes:
        # code that sends the new mode value to the right port
        eval('hub.port.' + str(port) + '.device.mode(' + str(mode) + ')')
        time.sleep(time_delay)
        val = eval('hub.port.' + str(port) + '.device.get()')
        if type(val) == list:
            sensor_data = val[0]
            if type(sensor_data) == int:
                average = average + sensor_data
                num_data_points = num_data_points + 1
            pass
        pass
    if num_data_points > 0:
        return average/num_data_points
    else:
        return -1 # didn't get any valid data, return -1

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
# Reads Distance Sensor (in Port C) 10 times (once every 1/2 second)
import time
import hub
for i in range(10):
    dist = avg_sensor('C',[0,1,2])
    print('Distance:', dist) # print to console
    hub.display.show(str(dist), fade=4) # show on Hub's LED screen
    time.sleep(0.5)
print('done')

## Code 10: Read Average Distance (MicroPython Code)

NOTE: must run on SPIKE Prime (Not in Jupyter Notebook)

Bonus code (using the `avg_sensor()` function from Code 9 above): display a little "rotating dial" (using the Clock Image) on the front of the SPIKE Prime based on the Distance Sensor.

- Distance Sensor plugged into Port C
- Force Sensor (for exiting) plugged into Port D

In [ ]:
# Note: to run on SPIKE Prime (NOT in Jupyter Notebook!)
# Reads Distance Sensor and display "clock" on front of SPIKE Prime showing relative distance
import time
import hub
val = hub.port.D.device.get() # get value of Force Sensor
force = val[0] # convert value to integer
while not force > 0: # loop until Force Sensor pushed
    dist = avg_sensor('C',[0,1,2]) # get distance from sensor in Port C
    if dist > 0:
        clock = dist/101 # percentage distance 0->100 (but don't want 100%)
        clock = clock * 12 # convert to 0->11
        hub.display.show(eval('hub.Image.CLOCK' + str(int(clock)+1)))
    else:
        # bad value (-1) came back from avg_sensor
        hub.display.show(hub.Image.SAD)
    # recalculate force value for next loop iteration
    val = hub.port.D.device.get() # get value of Force Sensor
    force = val[0] # convert value to integer
print('done')